![](img/cabecera.png)


# **Taller AWS: API REST en AWS EC2 - Extra Opcional**

En este Extra Opcional vamos a:

- **Crear un servicio** para que la API se reinicie automáticamente
- **Instalar nginx** como proxy reverso  
- **Eliminar la necesidad del puerto** en la URL
- **Descubrir las limitaciones** de esta aproximación
- **Evaluar alternativas** más prácticas

# Problema #1: Nuestra API se detiene al cerrar terminal

Actualmente, si lanzas Gunicorn y cierras la terminal SSH, **tu API se detiene.**  

Para solucionarlo, vamos a crear un **servicio** que mantenga tu API funcionando 24/7.

# Creación de un servicio systemd

## ¿Qué es un servicio?

Un **servicio** es un programa que se ejecuta en segundo plano y se gestiona automáticamente por el sistema operativo. Los servicios:

- **Se inician automáticamente** al arrancar el servidor
- **Se reinician solos** si fallan o se cuelgan
- **Funcionan sin terminal** - no necesitas estar conectado por SSH
- **Se gestionan fácilmente** con comandos estándar

**Sin servicio:** Si cierras la terminal, Gunicorn se detiene  
**Con servicio:** Gunicorn funciona 24/7 independientemente

## Creación del servicio

Para que Gunicorn se inicie automáticamente y se reinicie si falla, vamos a crear un servicio, abre tu GitBash o terminal y conéctate a tu instancia EC2 e introduce la siguiente secuencia:

```bash
ssh -i "flask-api-key.pem" ubuntu@ec2-XXX-XXX-XXX-XXX.compute-1.amazonaws.com
sudo nano /etc/systemd/system/flask-api.service
```

**Recuerda:** Sustituye la dirección DNS por la de tu instancia, que puedes encontrar en el panel de EC2.

El comando *sudo nano* creará un nuevo archivo de texto en tu instancia y lo abrirá usando *nano* como editor de texto.

**Contenido:**

> Copia y pega el siguiente contenido.


```ini
[Unit]
Description=Flask API with Gunicorn
After=network.target

[Service]
User=ubuntu
Group=ubuntu
WorkingDirectory=/home/ubuntu/tu-proyecto-flask
Environment="PATH=/home/ubuntu/tu-proyecto-flask/venv/bin"
ExecStart=/home/ubuntu/tu-proyecto-flask/venv/bin/gunicorn -w 3 -b 0.0.0.0:5000 app:app
Restart=always

[Install]
WantedBy=multi-user.target
```

> Recuerda, para guardar y salir de *nano*:  
>
> - Ctrl+O y Enter -> Guardar
> - Ctrl+X y Enter -> Salir

**IMPORTANTE: Cambia `tu-proyecto-flask` por el nombre real de tu directorio.**


Activamos el servicio:

```bash
sudo systemctl daemon-reload
sudo systemctl enable flask-api
sudo systemctl start flask-api
sudo systemctl status flask-api
```

Ahora puedes cerrar la terminal y la API seguirá funcionando. También puedes detener y reiniciar tu instancia y la API se levantará automáticamente.

# Problema #2: Solo podemos acceder a nuestra API usando el puerto 5000.

En la Parte 1 conseguimos desplegar nuestra API Flask en EC2 y acceder a ella usando `http://TU_IP_PUBLICA:5000`.  

Esto es más que suficiente pero no es una solución tan elegante como las que nos ofrecen plataformas PaaS como las vistas en otras sesiones.  

Podemos intentar mitigar este problema, pero verás que nos acabaremos encontrando con otros problemas...

Lo primero que vamos a hacer es instalar **nginx**.

# ¿Por qué necesitamos nginx?

Actualmente nuestra API funciona en `http://TU_IP_PUBLICA:5000` pero esto tiene limitaciones:

**Problemas actuales:**
- **Puerto visible** - Los usuarios deben recordar el `:5000`
- **Solo HTTP** - No hay HTTPS
- **Un solo proceso** - Gunicorn expuesto directamente
- **Sin archivos estáticos** - No puede servir CSS, JS, imágenes

**¿Qué es nginx?**

[nginx](https://nginx.org/) es un servidor web que actúa como **proxy reverso**. Recibe peticiones del exterior y las redirige internamente a nuestra aplicación, es decir, recibirá peticiones en el puerto 80 y las redirigirá internamente al puerto 5000 donde corre Gunicorn.

Por tanto, el flujo de tráfico con nginx sería:

```
Internet → nginx (puerto 80) → Gunicorn (puerto 5000) → Flask App
```

**Ventajas de usar nginx:**
- **Puerto 80 estándar** - `http://TU_IP_PUBLICA` (sin puerto)
- **HTTPS ready** - Fácil añadir certificados SSL
- **Balanceador** - Puede distribuir carga entre múltiples Gunicorn
- **Archivos estáticos** - Sirve CSS, JS e imágenes directamente
- **Seguridad** - Actúa como primera línea de defensa


# Instalación de nginx

Para instalar nginx, una vez conectados a nuestra instancia EC2 introduce la siguiente secuencia:

```bash
sudo apt install nginx -y
```

Verificamos que está funcionando:

```bash
sudo systemctl status nginx
```

Prueba en tu navegador: `http://TU_IP_PUBLICA` (sin puerto). Deberías ver la página de bienvenida de nginx.


<img src="img/nginx-welcome.png" alt="Nginx Welcome Page" style="width: 800px;">

# Configuración de nginx como Proxy

Para crear la configuración para redirigir el tráfico a nuestra API, introduce la secuencia:

```bash
sudo nano /etc/nginx/sites-available/flask-api
```

Este comando creará un nuevo archivo de texto en tu instancia y abrirá *nano* como editor de texto.

**Contenido del archivo:**

> Copia y pega el siguiente contenido.

```nginx
server {
    listen 80;
    server_name _;

    location / {
        proxy_pass http://0.0.0.0:5000;
        proxy_set_header Host $host;
        proxy_set_header X-Real-IP $remote_addr;
        proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
    }
}
```

> Para guardar y salir de *nano*:  
>
> - Ctrl+O y Enter -> Guardar
> - Ctrl+X y Enter -> Salir

Copia y ejecuta las siguientes sentencias en tu terminal para activar la configuración:

```bash
sudo ln -s /etc/nginx/sites-available/flask-api /etc/nginx/sites-enabled/
sudo rm /etc/nginx/sites-enabled/default
sudo nginx -t
sudo systemctl reload nginx
```

# Lanzamiento y prueba

Como ya tienes el servicio systemd configurado, tu API debería estar funcionando automáticamente. Si no es así, puedes reiniciar el servicio:

```bash
sudo systemctl restart flask-api
sudo systemctl status flask-api
```

Ahora prueba en tu navegador:  

```
http://TU_IP_PUBLICA
```

**¡Sin puerto!** Enhorabuena, deberías ver tu API Flask funcionando.

# Problema #3: IPs dinámicas

¡Perfecto! Pero hay un nuevo problema fundamental. Haz esta prueba:

1. **Apunta tu IP actual** (ej: `54.123.45.67`)
2. **Reinicia tu instancia** desde el panel de EC2
3. **Mira la nueva IP** después del reinicio

**¿Qué ha pasado?** La IP ha cambiado porque **AWS usa IPs dinámicas.**

**Problemas:**
- **Enlaces rotos** - URLs guardadas ya no funcionan
- **Clientes perdidos** - Apps que consumen tu API fallan
- **Sin continuidad** - Cada reinicio = nueva dirección

**¿Hay solución?**
- **AWS Elastic IP** ($5/mes)
- **Dominio + DNS dinámico** (configuración compleja)
- **Load Balancer** ($$)

Vaya, ¡qué mala pata!, pero espera que hay más...

# Problema #4: Cuando haga cambios en el proyecto, ¿tengo que actualizar la instancia?

Sí, así es, cada vez que modificas tu código local, necesitas actualizar la instancia EC2.

**¿Qué opciones tenemos para actualizar nuestro código?:**

**Opción #1: Manual (lo que harías ahora)**  

Conectando a través de GitBash o terminal a la instancia y usando:  

```bash
ssh -i "flask-api-key.pem" ubuntu@ec2-XXX-XXX-XXX-XXX.compute-1.amazonaws.com
cd tu-proyecto-flask
git pull origin main
sudo systemctl restart flask-api
```
> Pros:
> * Resuelve el problema... Hasta la próxima modificación (pues vaya pro).  
>
> Contras:  
> * Manual y repetitivo  
> * Propenso a errores  
> * Downtime durante la actualización  

**Opción #2: CI/CD Pipeline**  

Usando GitHub Actions + SSH automático

> Pros:
> * Automático en cada push
>
> Contras:
> * Configuración compleja
> * Gestión de secretos y claves

**Otras opciones**  

Existen otras alternativas como usar Docker + Registry (con una complejidad añadida considerable), pero te invitamos a que investigues por tu cuenta.  

Como verás, para proyectos sencillos esto se vuelve muy complejo de manejar, pero recuerda que...  

**En PaaS:** `git push` → deploy automático. **¡Así de simple!**  

# Moraleja del Taller: Plataformas PaaS

Para la mayoría de proyectos sencillos o en fases iniciales, las **Platform as a Service** comentadas en clase son mucho más prácticas porque **RESUELVEN TODOS ESTOS PROBLEMAS**:

- **[Render](https://render.com/)**
- **[Fly.io](https://fly.io/)**
- **[Vercel](https://vercel.com/)**
- **[Railway](https://railway.com/)**
- **[Heroku](https://www.heroku.com/)**
- **[PythonAnywhere](https://www.pythonanywhere.com/)**

En caso de no querer abandonar el ecosistema de AWS, una buena alternativa es **[AWS Elastic Beanstalk](https://aws.amazon.com/es/elasticbeanstalk/)**.

# Conclusión

## **Lo que has aprendido:**
- **Servicios systemd** - Automatización de servicios
- **Nginx como proxy reverso**  
- **Configuración de producción**  
- **Limitaciones de IaaS vs PaaS en proyectos sencillos**  

## **¿Cuándo usar cada opción?**

**Usa EC2 + nginx cuando necesites:**
- Control total necesario
- Configuraciones muy específicas
- Presupuesto para DevOps

**Usa PaaS cuando necesites:**
- Quieres enfocarte en desarrollo
- Proyecto en fase MVP
- Equipo pequeño
- Deploy rápido prioritario

## **Recomendación:**
Para la mayoría de proyectos, **empieza con PaaS**. Migra a infraestructura propia solo cuando sea realmente necesario.

**El tiempo que ahorras en infraestructura lo inviertes en features que generen valor.**

## **Y RECUERDA**

#### **¡Apaga tu instancia cuando no la uses para conservar tu Capa Gratuita o Free Tier!**


---

## Comandos de referencia

```bash
# Gestión del servicio
sudo systemctl status flask-api
sudo systemctl restart flask-api
sudo journalctl -u flask-api

# Nginx
sudo systemctl reload nginx
sudo nginx -t

# Debugging
sudo netstat -tlnp | grep :80
curl -I http://localhost
```